In [1]:
import logging
import json
import boto3
from botocore.exceptions import ClientError

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def list_foundation_models(bedrock_client):
    """
    Gets a list of available Amazon Bedrock foundation models.
    :return: The list of available bedrock foundation models.
    """
    try:
        response = bedrock_client.list_foundation_models()
        models = response["modelSummaries"]
        logger.info("Got %s foundation models.", len(models))
        return models
    except ClientError:
        logger.error("Couldn't list foundation models.")
        raise

def main():
    """Entry point for the example. Change aws_region to the AWS Region
    that you want to use."""
    
    aws_region = "us-east-1"
    bedrock_client = boto3.client(service_name="bedrock", region_name=aws_region)
    
    fm_models = list_foundation_models(bedrock_client)
    for model in fm_models:
        print(model["modelName"])
        print(json.dumps(model, indent=2))
        print("---------------------------\n")
        
    #print([x for x in fm_models])
    logger.info("Done.")

if __name__ == "__main__":
    main()

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:__main__:Got 80 foundation models.
INFO:__main__:Done.


Titan Text Large
{
  "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large",
  "modelId": "amazon.titan-tg1-large",
  "modelName": "Titan Text Large",
  "providerName": "Amazon",
  "inputModalities": [
    "TEXT"
  ],
  "outputModalities": [
    "TEXT"
  ],
  "responseStreamingSupported": true,
  "customizationsSupported": [],
  "inferenceTypesSupported": [
    "ON_DEMAND"
  ],
  "modelLifecycle": {
    "status": "ACTIVE"
  }
}
---------------------------

Titan Image Generator G1
{
  "modelArn": "arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0",
  "modelId": "amazon.titan-image-generator-v1:0",
  "modelName": "Titan Image Generator G1",
  "providerName": "Amazon",
  "inputModalities": [
    "TEXT",
    "IMAGE"
  ],
  "outputModalities": [
    "IMAGE"
  ],
  "customizationsSupported": [
    "FINE_TUNING"
  ],
  "inferenceTypesSupported": [
    "PROVISIONED"
  ],
  "modelLifecycle": {
    "status": "ACTIVE"
  }
}
------------------

In [5]:
# Use the Conversation API to send a text message to Amazon Titan Text G1 - Express.
import boto3
from botocore.exceptions import ClientError
# Create an Amazon Bedrock Runtime client.
brt = boto3.client("bedrock-runtime")
# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"
# Start a conversation with the user message.
#user_message = "Describe the purpose of a 'hello world' program in one line."
user_message = """Meeting transcript: Miguel: Hi Brant, I want to discuss the workstream  
            for our new product launch Brant: Sure Miguel, is there anything in particular you want
            to discuss? Miguel: Yes, I want to talk about how users enter into the product.
            Brant: Ok, in that case let me add in Namita. Namita: Hey everyone 
            Brant: Hi Namita, Miguel wants to discuss how users enter into the product.
            Miguel: its too complicated and we should remove friction.  
            for example, why do I need to fill out additional forms?  
            I also find it difficult to find where to access the product
            when I first land on the landing page. Brant: I would also add that
            I think there are too many steps. Namita: Ok, I can work on the
            landing page to make the product more discoverable but brant
            can you work on the additonal forms? Brant: Yes but I would need 
            to work with James from another team as he needs to unblock the sign up workflow.
            Miguel can you document any other concerns so that I can discuss with James only once?
            Miguel: Sure.
            From the meeting transcript above, Create a list of action items for each person. """


user_message = """Please classify the following customer support tickets into one of the categories: "Technical Issue," "Billing Inquiry," or "General Inquiry."

Use these examples to guide your classification:

Ticket: "I can't log in to my account. The system keeps giving me an error message."

Category: Technical Issue

Ticket: "I received an email saying my account will be suspended unless I make a payment. Is this a scam?"

Category: Billing Inquiry

Ticket: "What are the hours of operation for your customer support team?"

Category: General Inquiry

Now, classify the following ticket:

Ticket: "Can you explain why my latest bill is higher than usual? I haven't changed my plan."

"""

user_message = """Can you solve this by using the tree-of-thought technique?

Context:
- Bob is in the living room.
- He walks to the kitchen, carrying a cup.
- He puts a ball in the cup and carries the cup to the bedroom.
- He turns the cup upside down, then walks to the garden.
- He puts the cup down in the garden, then walks to the garage.
Question: Where is the ball?



"""


user_message = """Context: Amazon.com key facts: - Amazon.com was founded by Jeff Bezos in 1994. 
- Amazon.com started as an e-commerce platform but has since diversified into 
various other business segments. 
- Amazon.com operates numerous fulfillment centers and warehouses across the globe. 
 Instructions: Think step-by-step for providing a summary for Amazon.com based on the 
information shared in the Context before. 
Include your thoughts in the <thinking></thinking> XML tags and the actual summary as 
an Answer.



"""
conversation = [
    {
     "role": "user",
     "content": [{"text": user_message}],
    }
]
try:
    # Send the message to the model, using a basic inference configuration.
    response = brt.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )
    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)
except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

<thinking></thinking>
Amazon.com is an e-commerce platform founded by Jeff Bezos in 1994, offering a wide range of products and services. It began as an online bookstore and has since diversified into various other business segments, including digital streaming, cloud computing, and artificial intelligence. Amazon.com operates numerous fulfillment centers and warehouses globally to ensure efficient order processing and delivery.


In [3]:
import boto3
import botocore
import json 

bedrock_runtime = boto3.client('bedrock-runtime')

# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Command: Write me a blog about making strong business decisions as a leader.

Blog:
"""

try:

    '''
    body = json.dumps(
        {
            "messages": [{"role": "user", "content": [{"text": prompt_data}]}],
            "inferenceConfig": {
                "max_new_tokens": 500,
                "top_p": 0.9,
                "top_k": 20,
                "temperature": 0.7
            }
        }
    )
    '''
    body = json.dumps(
        {
            "inputText": prompt_data,
            "textGenerationConfig": {"maxTokenCount": 1024, "topP": 0.95, "temperature": 0.1},
        }
    )
    modelId = "amazon.titan-text-express-v1" # amazon.nova-lite-v1:0
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    outputText = response_body.get("results")[0].get("outputText")
    print(outputText)
    #response_body = json.loads(response.get("body").read())

    #print(response_body["output"]["message"]["content"][0]['text'])

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error

As a leader, making strong business decisions is crucial to the success of your organization. It requires careful consideration of various factors, including market trends, customer needs, and financial performance. Here are some tips to help you make effective business decisions:

Define Your Goals:

Before making any decision, it is important to define your goals. What do you want to achieve in the short-term and long-term? What are your long-term objectives? Once you have defined your goals, you can create a plan to achieve them.

Research Your Options:

Before making a decision, it is important to research your options. This includes gathering information from various sources, such as industry reports, customer feedback, and financial statements. You should also consider the potential risks and benefits of each option.

Consider the Impact:

Every decision you make has an impact on your organization. It is important to consider the impact of your decision on your employees, custome